# Experiments on Random Osborne Algorithm for Matrix Balancing

In [3]:
import osborn
import timeit
import np


KeyboardInterrupt



## 1. Near-linear convergence

In [ ]:
variants = ['cyclic', 'random_cyclic', 'greedy', 'random']
epsilon_values = [10**(-i) for i in range(10)]
n_values = [10, 25, 50, 100, 200]
non_zero_ratios = [.1, .2, .3, .5, .8]

execution_time = dict()


for variant in variants:
    for n in n_values:
        for non_zero_ratio in non_zero_ratios:
            m = n*non_zero_ratio
            K = np.random.randn(n,n)